# Madrid South Real Estate Investment Analysis

This notebook demonstrates how to analyze the processed real estate data to identify investment opportunities.

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

In [ ]:
# Connect to BigQuery
project_id = os.getenv('BIGQUERY_PROJECT_ID')
client = bigquery.Client(project=project_id)

# Query the mart table
query = """
SELECT 
    property_code,
    address,
    municipality,
    price,
    size,
    rooms,
    price_by_area,
    price_deviation_zone_pct,
    competitiveness_index,
    opportunity_classification,
    premium_features_count,
    avg_price_zone,
    avg_price_m2_zone
FROM `{}.dbt_mart.madrid_south_real_estate_metrics`
WHERE is_valid_record = true
""".format(project_id)

df = client.query(query).to_dataframe()
print(f"Loaded {len(df)} properties for analysis")

## 1. Investment Opportunities Overview

In [ ]:
# Opportunity distribution
opportunity_dist = df['opportunity_classification'].value_counts()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Pie chart
colors = ['#2ecc71', '#95a5a6', '#e74c3c']
ax1.pie(opportunity_dist.values, labels=opportunity_dist.index, autopct='%1.1f%%', colors=colors)
ax1.set_title('Property Classification Distribution')

# Bar chart with counts
opportunity_dist.plot(kind='bar', ax=ax2, color=colors)
ax2.set_title('Number of Properties by Classification')
ax2.set_xlabel('Classification')
ax2.set_ylabel('Count')
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=0)

plt.tight_layout()
plt.show()

## 2. Top Investment Opportunities

In [ ]:
# Find top opportunities
opportunities = df[df['opportunity_classification'] == 'Opportunity'].copy()
opportunities = opportunities.sort_values('competitiveness_index', ascending=False).head(10)

# Display key metrics
display_cols = [
    'address', 'municipality', 'price', 'size', 
    'price_deviation_zone_pct', 'competitiveness_index', 
    'premium_features_count'
]

print("\n🎯 TOP 10 INVESTMENT OPPORTUNITIES IN SOUTHERN MADRID\n")
print(opportunities[display_cols].to_string(index=False))

## 3. Price Analysis by Zone

In [ ]:
# Price deviation analysis
fig, ax = plt.subplots(figsize=(14, 8))

# Group by municipality and calculate average price deviation
zone_analysis = df.groupby('municipality').agg({
    'price_deviation_zone_pct': 'mean',
    'property_code': 'count',
    'price': 'mean'
}).round(2)

zone_analysis.columns = ['avg_price_deviation', 'property_count', 'avg_price']
zone_analysis = zone_analysis.sort_values('avg_price_deviation')

# Create bar plot
colors = ['green' if x < 0 else 'red' for x in zone_analysis['avg_price_deviation']]
zone_analysis['avg_price_deviation'].plot(kind='barh', color=colors, ax=ax)

ax.set_title('Average Price Deviation by Municipality', fontsize=14, fontweight='bold')
ax.set_xlabel('Price Deviation from Zone Average (%)')
ax.set_ylabel('Municipality')
ax.axvline(x=0, color='black', linestyle='-', linewidth=0.5)

# Add value labels
for i, (v, count) in enumerate(zip(zone_analysis['avg_price_deviation'], zone_analysis['property_count'])):
    ax.text(v, i, f' {v}% ({count} props)', va='center')

plt.tight_layout()
plt.show()

## 4. Premium Features Analysis

In [ ]:
# Analyze relationship between features and opportunity classification
feature_analysis = df.groupby(['premium_features_count', 'opportunity_classification']).size().unstack(fill_value=0)

fig, ax = plt.subplots(figsize=(12, 6))
feature_analysis.plot(kind='bar', stacked=True, ax=ax, color=['#2ecc71', '#95a5a6', '#e74c3c'])

ax.set_title('Property Classification by Premium Features Count', fontsize=14, fontweight='bold')
ax.set_xlabel('Number of Premium Features')
ax.set_ylabel('Number of Properties')
ax.legend(title='Classification')
ax.set_xticklabels(ax.get_xticklabels(), rotation=0)

plt.tight_layout()
plt.show()

## 5. Investment Recommendations

In [ ]:
# Generate investment recommendations
print("\n📊 INVESTMENT INSIGHTS FOR SOUTHERN MADRID\n")
print("="*60)

# Opportunities summary
opp_pct = (len(df[df['opportunity_classification'] == 'Opportunity']) / len(df)) * 100
print(f"\n✅ Investment Opportunities: {opp_pct:.1f}% of market")

# Best municipality for investment
best_municipality = df[df['opportunity_classification'] == 'Opportunity']['municipality'].value_counts().head(1)
if not best_municipality.empty:
    print(f"🏆 Top Municipality: {best_municipality.index[0]} ({best_municipality.values[0]} opportunities)")

# Average discount for opportunities
avg_discount = df[df['opportunity_classification'] == 'Opportunity']['price_deviation_zone_pct'].mean()
print(f"💰 Average Discount: {avg_discount:.1f}% below zone average")

# Sweet spot for features
sweet_spot = df[df['opportunity_classification'] == 'Opportunity']['premium_features_count'].mode()
if not sweet_spot.empty:
    print(f"🎯 Optimal Features Count: {sweet_spot.values[0]} premium features")

print("\n" + "="*60)
print("\n💡 KEY RECOMMENDATIONS:")
print("1. Focus on properties 20%+ below zone average")
print("2. Prioritize properties with 2-3 premium features")
print("3. Target undervalued municipalities identified above")
print("4. Act quickly on properties with high competitiveness index")